## GeoCat experimental

In [1]:
import importlib
from collections import defaultdict
import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
from IPython.core.debugger import set_trace
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
importlib.reload(geocat)
importlib.reload(rec)
CITY="madison"



In [2]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})

df_checkin_train=pd.read_csv("../data/checkin/train/"+CITY+".csv",converters={'categories':string_to_array})


users_id=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
pois_id=df_checkin['business_id'].drop_duplicates().reset_index(drop=True)
user_num=len(users_id)
poi_num=len(pois_id)


df_checkin_train=df_checkin_train.set_index('user_id')

df_checkin_train.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
0,1744,2017-12-15 14:05:01,43.075012,-89.447498,"[desserts, cafes, coffee]",2143.0
0,2746,2018-01-18 20:57:23,43.106774,-89.497738,"[desserts, tradamerican, diners, gastropubs, b...",2641.0
0,30,2018-01-23 23:14:37,43.072654,-89.384062,"[bars, mexican]",2133.0
0,432,2018-01-27 02:17:06,43.072439,-89.384610,"[tradamerican, bars]",2133.0
0,22,2018-01-27 02:31:26,43.075100,-89.381069,[mexican],2132.0


In [3]:
df_checkin_test=pd.read_csv("../data/checkin/test/"+CITY+".csv",converters={'categories':string_to_array})

In [4]:
ground_truth = defaultdict(set)
for index,row in df_checkin_test[['user_id','business_id']].drop_duplicates().iterrows():
    ground_truth[row['user_id']].add(row['business_id'])

In [5]:
poi_coos = {}
for lid,latlon in df_checkin[['business_id','latitude','longitude']].reset_index(drop=True).drop_duplicates().set_index('business_id').iterrows():
    poi_coos[lid] = tuple(latlon)

In [6]:
poi_cats = {}
for lid,row in df_checkin[['business_id','categories']].reset_index(drop=True).drop_duplicates(subset=['business_id']).set_index('business_id').iterrows():
    poi_cats[lid] = row.categories

In [7]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [8]:
training_matrix = np.zeros((user_num, poi_num))
for user_id,poi_id in df_checkin_train.business_id.iteritems():
   # print(user_id,poi_id)
    training_matrix[user_id,poi_id]+=1

In [9]:
all_uids = list(range(user_num))
all_lids = list(range(poi_num))

In [10]:
#np.random.shuffle(all_uids)

In [11]:
user_predict={}
for index,row in pd.read_csv("../data/result/sigir11_top_100.txt",header=None).iterrows():
    user_id=row[0]
    lid_list=row[1]
    user_predict[user_id]=eval(lid_list)

In [12]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")
poi_neighbors={}
for lid,row in df_poi_neighbor.iterrows():
    neighbors=row['neighbors']
    poi_neighbors[lid]=neighbors
df_poi_neighbor=None

In [13]:
import objfunction
importlib.reload(objfunction)

<module 'objfunction' from '/home/heitor/recsys/algorithms/objfunction.py'>

In [14]:
#lvisited=set(training_matrix[uid].nonzero()[0])

In [111]:
importlib.reload(rec)
importlib.reload(objfunction)
N=80
K=20
range_K=range(K)
overall_scores=np.zeros(poi_num)
for cnt, uid in enumerate(all_uids):
    if uid in ground_truth:
        
        print("userid:",uid)
        if uid==0:
            continue
        
        pois_score=rec.mostpopularnp(training_matrix,uid)
        tmp_rec_list=list(reversed(np.argsort(pois_score)))[0:N]
        tmp_score_list=list(reversed(np.sort(pois_score)))[0:N]
        #print(list(reversed(np.sort(pois_score)))[0:N])
        
        #print(tmp_rec_list)
        #break
        #tmp_rec_list=user_predict[uid][:N]
        rec_list=[]
        
        lids=training_matrix[uid].nonzero()[0]
        
        lid_visits=training_matrix[:,lids].sum(axis=0)
        mean_visits=lid_visits.mean()
        relevant_lids=lids[lid_visits>mean_visits]
        relevant_cats=set()
        for lid in relevant_lids:
            relevant_cats.update(poi_cats[lid])
        # log_size=training_matrix[0,training_matrix[0,:].nonzero()[0]].sum()
        user_log=training_matrix[uid]
        print(mean_visits)
        log_poi_ids=list()
        poi_cover=list()
        for lid in user_log.nonzero()[0]:
            for visits in range(int(user_log[lid])):
                poi_cover.append(0)
                log_poi_ids.append(lid)
                
        assert user_log[user_log.nonzero()[0]].sum() == len(poi_cover)
#         print(uid)
#         print("Count:",cnt)
        div_geo_cat_weight = 0.75 # beta,this is here because of the work to be done on parameter customization for each user
        div_weight = 0.5 # lambda, geo vs cat
        current_proportionality=0
        final_scores=[]
        for i in range_K:
            print(i)
            poi_to_insert=None
            max_objective_value=-200
            for j in range(len(tmp_rec_list)):
                candidate_poi_id=tmp_rec_list[j]
                candidate_score=tmp_score_list[j]
                ild_div=objfunction.min_dist_to_list_cat(candidate_poi_id,rec_list,poi_cats,undirected_category_tree)
                gc_div=objfunction.gc(candidate_poi_id,rec_list,relevant_cats,poi_cats)
                pr=objfunction.update_geo_cov(candidate_poi_id,log_poi_ids,K,poi_cover.copy(),poi_neighbors)
                
                objective_value=objfunction.ILD_GC_PR(candidate_score,ild_div,gc_div,pr,current_proportionality,K,div_geo_cat_weight,div_weight)
                print(candidate_poi_id,ild_div,gc_div,max(0,pr-current_proportionality),objective_value)
                #print(candidate_poi_id,objective_value)

                if objective_value > max_objective_value:
                    max_objective_value=objective_value
                    poi_to_insert=candidate_poi_id
                pass
            if poi_to_insert is not None:
                print(poi_to_insert,max_objective_value)
                
                rm_idx=tmp_rec_list.index(poi_to_insert)
                
                tmp_rec_list.pop(rm_idx)
                tmp_score_list.pop(rm_idx)
                rec_list.append(poi_to_insert)
                final_scores.append(max_objective_value)
                # remove from tmp_rec_list
                current_proportionality=objfunction.update_geo_cov(poi_to_insert,log_poi_ids,K,poi_cover,poi_neighbors)
                #print(current_proportionality)
#         print(final_scores)
#         print(rec_list)
        print(list(reversed(np.sort(final_scores))))
        print(np.array(rec_list)[list(reversed(np.argsort(final_scores)))])
        #print()
        break

userid: 0
userid: 1
68.05
0
2326 1.0 0.16 0.1022922983626442 0.14704022927902316
10 1.0 0.08 0.010283505154639161 0.08178624056651826
343 1.0 0.12 0.10136082474226815 0.1353630247361261
361 1.0 0.0 0.030927835051546393 0.07189760665135797
2332 1.0 0.08 0.1038800374882849 0.11850876942089711
1338 1.0 0.04 0.1022922983626442 0.11060066473752084
9 1.0 0.08 0.010283505154639161 0.06697430423170214
1040 1.0 0.08 0.10452783505154661 0.11024052224097423
2658 1.0 0.08 0.09740206185567035 0.10561332850004498
2675 1.0 0.12 0.09231222385861582 0.10866243413219735
3006 1.0 0.08 0.030927835051546393 0.07636211729525756
1688 1.0 0.0 0.010283505154639161 0.04557016515203979
2018 1.0 0.0 0.09608247422680416 0.09320870068783656
68 1.0 0.12 0.010283505154639161 0.07009403026551513
682 1.0 0.08 0.010283505154639161 0.06209674313276544
2688 1.0 0.08 0.05154639175257736 0.0826047327495033
1017 1.0 0.04 0.030927835051546393 0.0658565421862398
2692 1.0 0.04 0.030927835051546393 0.06551619602059584
3029 1.0 0

2022 0.0 0.24 0 0.06177416099577475
2067 0.0 0.2 0 0.05639183574723604
42 0.6666666666666667 0.2 0 0.060439788521549936
2746 0.0 0.28 0 0.06516575283433587
676 0.0 0.2 0.06497028502122482 0.07739090707905887
1666 0.0 0.24 0.0646442692540935 0.0811244273968397
1683 0.6666666666666667 0.24 0 0.06438529650269834
1073 0.0 0.24 0 0.06033176379083826
1715 0.6666666666666667 0.28 0 0.06837742996704092
374 0.0 0.24 0 0.0593506941186698
2371 0.0 0.2 0.08690225481007785 0.08223041386992219
2317 0.0 0.24 0 0.058353132414826786
1723 0.6666666666666667 0.24 0.020618556701030966 0.06896098155006578
1371 0.6666666666666667 0.24 0.05786779866585812 0.07963089124705656
1418 0.0 0.2 0 0.05326887820781089
19 0.8 0.2 0 0.05835313241482679
2701 0.0 0.28 0.08200294550810028 0.0856403639520456
2014 0.6666666666666667 0.24 0.08200294550810028 0.08434616944129499
1028 0.0 0.2 0 0.05234239225902136
2359 0.6666666666666667 0.24 0.059381443298969216 0.07864239011946375
693 0.0 0.24 0.06497028502122482 0.076493762

1381 0.0 0.28 0 0.06924247647178143
1108 0.6666666666666667 0.28 0.06923207926094532 0.09240876029178836
375 0.0 0.36 0.044278868735873056 0.08920064676883344
1046 0.0 0.28 0.06160851204586815 0.08662864344566003
3013 0.0 0.28 0 0.06672371987921948
2022 0.0 0.28 0 0.06672371987921948
2067 0.0 0.28 0 0.06672371987921948
42 0.6666666666666667 0.28 0 0.07003866232648018
2746 0.0 0.36 0 0.07389101828483793
676 0.0 0.28 0.045866607861513975 0.07958304543620197
1666 0.0 0.32 0.052465675064777506 0.08509033929882058
1683 0.6666666666666667 0.32 0 0.07320364503523633
1073 0.0 0.32 0 0.06966512013065078
1715 0.0 0.28 0 0.06463808623235083
374 0.0 0.32 0 0.06853227845201029
2371 0.0 0.28 0.07139437375018176 0.08516558534042883
2317 0.0 0.28 0 0.06302858668672516
1723 0.6666666666666667 0.32 0 0.07080286939394602
1371 0.0 0.32 0.05786779866585812 0.08368500043270331
1418 0.0 0.28 0 0.06302858668672516
19 0.8 0.28 0 0.06738039341551624
2701 0.0 0.36 0.06923207926094532 0.0889692628622485
2014 0.0 

682 0.6666666666666667 0.4 0 0.10192216566518432
2688 0.0 0.36 0.020618556701030966 0.10056330539954603
1017 0.0 0.32 0 0.08713756360040865
2692 0.0 0.36 0.020618556701030966 0.09953185961661676
3029 0.0 0.32 0.03189450355587409 0.09776645812013088
2313 0.6666666666666667 0.32 0 0.090136520360365
1363 0.0 0.32 0 0.08393455655887992
1048 0.6666666666666667 0.32 0.034040309771617316 0.09957766660658529
2717 0.0 0.36 0 0.08752969862224018
2666 0.0 0.32 0.03627186504217428 0.09497932257832035
1662 0.0 0.32 0.04123711340206193 0.096614886107912
2010 0.0 0.32 0 0.08204814585807257
694 0.6666666666666667 0.32 0.03394009268297438 0.09728211764138124
2392 0.0 0.32 0.020618556701030966 0.0891053214288467
1388 0.0 0.36 0 0.08651776058291037
371 0.0 0.32 0 0.08108848540793831
2319 0.6666666666666667 0.32 0 0.08469853682086582
2724 0.0 0.44 0.011088201603665526 0.09623688614361091
743 0.0 0.32 0.04316553598009043 0.09378918442065175
16 0.0 0.32 0 0.07913425882489307
704 0.0 0.36 0 0.083408315906908

3035 0.0 0.48 2.577319587615534e-05 0.0957078262006078
1712 0.0 0.48 0.020644329896907232 0.10103287548667543
2357 0.0 0.48 0 0.09446522672743322
2049 0.0 0.48 0.029934947587282323 0.10224409992970283
2366 0.0 0.48 0 0.09258200997725514
2006 0.0 0.48 0.03159594113184172 0.10061542923389626
3015 0.0 0.48 2.577319587615534e-05 0.09195310508520885
2672 0.0 0.48 0 0.09130495557163026
1381 0.0 0.48 0 0.0906596827823249
1108 0.6666666666666667 0.48 0.01929802364173061 0.09675790911269913
375 0.0 0.56 0.03133054507651223 0.10275491894410096
1046 0.0 0.48 0.03104227661698844 0.096192651059975
3013 0.0 0.48 0 0.0873618562844437
2022 0.0 0.48 0 0.0873618562844437
2067 0.0 0.48 0 0.0873618562844437
42 0.6666666666666667 0.48 0 0.0896466936387801
2746 0.0 0.52 0 0.08880595174074347
676 0.0 0.48 0.03192027675175013 0.0934461599172707
1666 0.0 0.52 0.04271097524670586 0.0991453712824934
1683 0.0 0.48 0 0.08532199859489349
1073 0.0 0.48 0 0.08532199859489349
1715 0.0 0.48 0 0.08463112083909446
374 0.

371 0.0 0.52 0 0.10336794235569031
2319 0.6666666666666667 0.52 0 0.10599295967955037
2724 0.0 0.56 0.000492640905159325 0.10482283280007486
743 0.0 0.52 0.021039830968688777 0.10682387546790151
16 0.0 0.52 0 0.10087678248545796
704 0.0 0.56 0 0.10402844711048598
2758 0.0 0.52 0 0.10024431993415221
1050 0.0 0.56 0 0.10336794235569033
2344 0.0 0.52 0.018293765392929262 0.10473237385149745
373 0.0 0.56 0 0.10336794235569033
3035 0.0 0.52 2.5773195876266364e-05 0.09961524677172735
1712 0.0 0.52 0.020644329896907232 0.10469511252508547
2357 0.0 0.52 0 0.09832252530517856
2049 0.0 0.52 0.019214656045736023 0.10294501221855161
2366 0.0 0.52 0 0.09636241116594317
2006 0.0 0.52 0.023702358080626817 0.10203375816195107
3015 0.0 0.52 2.5773195876266364e-05 0.0957072333383692
2672 0.0 0.52 0 0.09503321079811429
1381 0.0 0.52 0 0.09436158958517547
1108 0.0 0.52 0.0006762072141570652 0.09249731266266849
375 0.0 0.6 0.03133054507651234 0.10585067751700765
1046 0.0 0.52 0.022116689536420653 0.0972960

2666 0.0 0.52 0.00928355397958125 0.10735564299039707
1662 0.0 0.52 0.020618556701030855 0.11063726147828042
2010 0.0 0.52 0 0.10459127419610478
694 0.6666666666666667 0.52 0.007030108403357271 0.1092874212984188
2392 0.0 0.52 0 0.1039814073400002
1388 0.0 0.56 0 0.10790660599072867
371 0.0 0.52 0 0.10336794235569031
2319 0.6666666666666667 0.52 0 0.10599295967955037
2724 0.0 0.56 0.00046686770928305865 0.10481561511891416
743 0.0 0.52 0.008347476636425322 0.10327725967574201
16 0.0 0.52 0 0.10087678248545796
704 0.0 0.56 0 0.10402844711048598
2758 0.0 0.52 0 0.10024431993415221
1050 0.0 0.56 0 0.10336794235569033
2344 0.0 0.52 0.008424594565598298 0.10199976392900573
373 0.0 0.56 0 0.10336794235569033
3035 0.0 0.52 0 0.09960784162656541
1712 0.0 0.52 0.020618556701030855 0.10468815704053899
2357 0.0 0.52 0 0.09832252530517856
2049 0.0 0.52 0.008259472306587967 0.09997355480680856
2366 0.0 0.52 0 0.09636241116594317
2006 0.0 0.52 0.007516746085418191 0.09775315515657358
3015 0.0 0.52 0